In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold, StratifiedKFold
from keras import models, layers
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import average_precision_score
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import gc
import matplotlib.pyplot as plt
%matplotlib inline
    
GAUSSIAN_NOISE = 0.05
UPSAMPLE_MODE = 'SIMPLE'
IMG_SIZE = (299, 299) # [(224, 224), (384, 384), (512, 512), (640, 640)]
IMG_SHAPE = (299, 299, 3)
BATCH_SIZE = 64

DENSE_COUNT = 128
LEARN_RATE = 1e-4
RGB_FLIP = 1

Using TensorFlow backend.


In [2]:
test_df = pd.read_csv('../input/sample_submission.csv.zip')
test_df['path'] = '../input/test/' + test_df.ImageId
test_mask = pd.read_csv('../input/test_mask.csv')
#img_test = np.load('../input/img_test.npy')

In [3]:
unique = pd.read_csv('../input/unique_img_ids.csv')
unique.reset_index(inplace=True)

In [4]:
def load_img(file, preprocess_func, IMG_SIZE = IMG_SIZE):
    if os.path.exists(file):
        img = cv2.imread(file)
        return preprocess_func(cv2.resize(img, IMG_SIZE))
    else:
        print(file, 'not found')
        return None

In [5]:
def gen_cache(df, preprocess_func):
    print('gen cache')
    cnt = len(df)
    img_all = np.zeros((cnt, 299,299,3), dtype=np.int8)
    from tqdm import tqdm_notebook
    prog = tqdm_notebook(total = cnt)
    for idx in range(cnt):
        row = df.iloc[idx]
        img_all[idx] = load_img(row.path, preprocess_func)
        prog.update(1)
        
    return img_all

In [6]:
def check_cache(df, preprocess_func, tot_fold, n_fold, flex = 'img'):
    savepath = '../cache/sub_{}_{}_{}.npy'.format(flex,tot_fold, n_fold)
    if os.path.exists(savepath):
        print('load cache from file')
        return np.load(savepath)
    else:
        cache = gen_cache(df, preprocess_func)
        np.save(savepath, cache)
        return cache

In [7]:
def get_model(BASE_MODEL):
    
    from keras.preprocessing.image import ImageDataGenerator
    if BASE_MODEL=='VGG16':
        from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
    elif BASE_MODEL=='RESNET52':
        from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
    elif BASE_MODEL=='InceptionV3':
        from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
    elif BASE_MODEL=='Xception':
        from keras.applications.xception import Xception as PTModel, preprocess_input
    elif BASE_MODEL=='DenseNet169': 
        from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
    elif BASE_MODEL=='DenseNet121':
        from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
    else:
        raise ValueError('Unknown model: {}'.format(BASE_MODEL))
        
    base_pretrained_model = PTModel(input_shape =  IMG_SHAPE, 
                                  include_top = False, weights = 'imagenet')
    base_pretrained_model.trainable = False
    
    from keras import models, layers
    from keras.optimizers import Adam
    img_in = layers.Input(IMG_SHAPE, name='Image_RGB_In')
    img_noise = layers.GaussianNoise(GAUSSIAN_NOISE)(img_in)
    pt_features = base_pretrained_model(img_noise)
    pt_depth = base_pretrained_model.get_output_shape_at(0)[-1]
    bn_features = layers.BatchNormalization()(pt_features)
    feature_dropout = layers.SpatialDropout2D(0.5)(bn_features)#bn_features
    #feature_dropout = bn_features
    x = layers.GlobalMaxPooling2D()(feature_dropout)

    dr_steps = layers.Dropout(0.5)(layers.Dense(128, activation = 'relu')(x))
    out_layer = layers.Dense(1, activation = 'sigmoid')(dr_steps)

    ship_model = models.Model(inputs = [img_in], outputs = [out_layer], name = 'full_model')

    ship_model.compile(optimizer = Adam(lr=LEARN_RATE), 
                       loss = 'binary_crossentropy',
                       metrics = ['binary_accuracy'])
    
    return ship_model, preprocess_input

In [8]:
def get_callbacks(mname, n_fold, checkflex=''):
    from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
    weight_path="logs/boat_detecotr/best_{}{}.best.hdf5".format(mname, n_fold)

    checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, save_weights_only = True)

    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, verbose=1, mode='auto', epsilon=0.0001, cooldown=1, min_lr=0.000001)
    early = EarlyStopping(monitor="val_loss", 
                          #mode="min", 
                          patience=5) # probably needs to be more patient, but kaggle time is limited
    board = TensorBoard(log_dir='logs/boat_detecotr/{}_{}'.format(mname, checkflex),
                        histogram_freq=0, 
                        write_graph=True, write_images=False)
    callbacks_list = [checkpoint, early, reduceLROnPlat, board]
    return callbacks_list, weight_path

In [9]:
def get_imggenerator(preprocess_input):
    from keras.preprocessing.image import ImageDataGenerator
    dg_args = dict(featurewise_center = False, 
                      samplewise_center = False,
                      rotation_range = 45, 
                      width_shift_range = 0.05, 
                      height_shift_range = 0.05, 
                      shear_range = 0.01,
                      #zoom_range = [0.9, 1.25],  
                      brightness_range = [0.5, 1.5],
                      horizontal_flip = True, 
                      vertical_flip = True,
                      fill_mode = 'reflect',
                      data_format = 'channels_last',
                      preprocessing_function = preprocess_input)
    
    core_idg = ImageDataGenerator(**dg_args)
    return core_idg

In [10]:
def get_valid_idg(preprocess_input):
    from keras.preprocessing.image import ImageDataGenerator
    valid_args = dict(fill_mode = 'reflect',
                   data_format = 'channels_last',
                  preprocessing_function = preprocess_input)

    valid_idg = ImageDataGenerator(**valid_args)
    return valid_idg

In [11]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [12]:
def get_img_flow_func(train_df, val_df, preprocess_input):
    core_idg = get_imggenerator(preprocess_input)
    train_gen = flow_from_dataframe(core_idg, train_df, 
                                 path_col = 'path',
                                y_col = 'has_ship', 
                                target_size = IMG_SIZE,
                                 color_mode = 'rgb',
                                batch_size = BATCH_SIZE)
    
    valid_idg = get_valid_idg(preprocess_input)
    valid_x, valid_y = next(flow_from_dataframe(valid_idg, 
                               val_df, 
                             path_col = 'path',
                            y_col = 'has_ship', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = len(val_df))) # one big batch
    return train_gen, valid_x, valid_y

In [13]:
def predict_in_gen(ship_model, df, preprocess_input):
    INF_BATCH_SIZE = BATCH_SIZE * 4
    valid_idg = get_valid_idg(preprocess_input)
    test_gen = flow_from_dataframe(valid_idg, 
                               df, 
                             path_col = 'path',
                            y_col = 'ImageId', 
                            target_size = IMG_SIZE,
                             color_mode = 'rgb',
                            batch_size = INF_BATCH_SIZE, 
                              shuffle = False)
    from tqdm import tqdm_notebook
    all_scores = dict()
    for _, (t_x, t_names) in zip(tqdm_notebook(range(test_gen.n//BATCH_SIZE+1)),
                                test_gen):
        t_y = ship_model.predict(t_x)[:, 0].clip(0,1)
        for c_id, c_score in zip(t_names, t_y):
            all_scores[c_id] = c_score
            
    return df['ImageId'].map(lambda x: all_scores.get(x, 0))

In [14]:
model_lst = [
    'Xception',
    'InceptionV3',
    'RESNET52',
    'VGG16',
    'DenseNet169',
]

debug = 0

num_folds = 3
for mname in model_lst:
    
    oof_preds = np.zeros(unique.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(unique, unique['ships'])):
        print('Fold', n_fold)
        train_df = unique.iloc[train_idx]
        val_df = unique.iloc[valid_idx]
        
        if debug:
            train_df = train_df[:5000]
            val_df = val_df[:100]
            test_df = test_df[:100]
        gc.collect()
        
        val_folds = StratifiedKFold(n_splits= 10, shuffle=True, random_state=1001)
        _, hold_idx = next(val_folds.split(val_df, val_df['ships']))
        hold_df = val_df.iloc[hold_idx]
        
        callbacks_list, weight_path = get_callbacks(mname, n_fold, 
                                                    checkflex='fix')
        model, preprocess_input = get_model(mname)
        
        val_y = val_df['has_ship'].values
        hold_y = hold_df['has_ship'].values

        img_val = check_cache(val_df, preprocess_input, num_folds, n_fold, flex = mname)
        
        
        train_img_cnt = 100 if debug else 10000
        train_gen, img_hold, hold_y = get_img_flow_func(train_df, hold_df,
                                                        preprocess_input)
        
        if os.path.exists(weight_path):
            model.load_weights(weight_path)
            
        model.fit_generator(train_gen, 
                          validation_data = (img_hold, hold_y), 
                          epochs = 2 if debug else 300, 
                          callbacks = callbacks_list,
                          workers = 8,
                          max_queue_size=100,
                          steps_per_epoch = round(train_img_cnt / BATCH_SIZE),
                          #validation_steps = round(VALID_IMG_COUNT / BATCH_SIZE),
                          use_multiprocessing=True)
        
        
        print( 'Predicting...' )
        model.load_weights(weight_path)
        
        
        oof_preds[val_df['index']] = model.predict(img_val).flatten().clip(0,1)
        #oof_preds[val_df['index']] = predict_in_gen(model, val_df, preprocess_input)
        print('predict test')
        
        del img_val,img_hold,val_y,hold_y
        gc.collect()
        
        img_test = check_cache(test_df, preprocess_input, 0, 0, flex = mname)
        cur_pred = model.predict(img_test).flatten().clip(0,1) / num_folds
        #cur_pred = predict_in_gen(model, test_df, preprocess_input)
        sub_preds += cur_pred
        
        del model, img_test
        gc.collect()
        
        from keras import backend as K
        K.clear_session()
        
        if debug:
            break
            
    print('gen cache')
    unique['{}_oof'.format(mname)] = oof_preds
    unique['{}_oof'.format(mname)].to_csv('../result/{}_oof.csv'.format(mname))
    test_df['{}_pre'.format(mname)] = sub_preds
    test_df['{}_pre'.format(mname)].to_csv('../result/{}_pre.csv'.format(mname))
    
    del oof_preds, sub_preds
    gc.collect()
        

Fold 0


/home/kownse/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


load cache from file
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 69376 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 3477 images
Epoch 1/300
156/156 [==============================] - 103s 658ms/step - loss: 0.3670 - binary_accuracy: 0.8561 - val_loss: 0.3348 - val_binary_accuracy: 0.8617

Epoch 00001: val_loss improved from inf to 0.33480, saving model to logs/boat_detecotr/best_Xception0.best.hdf5
Epoch 2/300
156/156 [==============================] - 92s 588ms/step - loss: 0.3628 - binary_accuracy: 0.8536 - val_loss: 0.3515 - val_binary_accuracy: 0.8502

Epoch 00002: val_loss did not improve from 0.33480
Epoch 3/300
156/156 [==============================] - 92s 587ms/step - loss: 0.3577 - binary_accuracy: 0.8603 - val_loss: 0.3571 - val_binary_accuracy: 0.8427

Epoch 00003: val_loss did not improve from 0.33480

Epoch

/home/kownse/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


gen cache


...

## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 69384 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 3476 images
Epoch 1/300
156/156 [==============================] - 102s 655ms/step - loss: 2.0082 - binary_accuracy: 0.6919 - val_loss: 0.5428 - val_binary_accuracy: 0.8921

Epoch 00001: val_loss improved from inf to 0.54276, saving model to logs/boat_detecotr/best_VGG162.best.hdf5
Epoch 2/300
156/156 [==============================] - 94s 605ms/step - loss: 1.2914 - binary_accuracy: 0.8011 - val_loss: 0.5186 - val_binary_accuracy: 0.9054

Epoch 00002: val_loss improved from 0.54276 to 0.51859, saving model to logs/boat_detecotr/best_VGG162.best.hdf5
Epoch 3/300
156/156 [==============================] - 94s 601ms/step - loss: 1.0733 - binary_accuracy: 0.8270 - val_loss: 0.4892 - val_binary_accuracy: 0.9100

Epoch 00003: val_lo

## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 69376 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 3477 images
Epoch 1/300
156/156 [==============================] - 106s 678ms/step - loss: 1.0953 - binary_accuracy: 0.6549 - val_loss: 0.5186 - val_binary_accuracy: 0.7760

Epoch 00001: val_loss improved from inf to 0.51861, saving model to logs/boat_detecotr/best_DenseNet1690.best.hdf5
Epoch 2/300
156/156 [==============================] - 90s 579ms/step - loss: 0.6036 - binary_accuracy: 0.7258 - val_loss: 0.5015 - val_binary_accuracy: 0.7777

Epoch 00002: val_loss improved from 0.51861 to 0.50155, saving model to logs/boat_detecotr/best_DenseNet1690.best.hdf5
Epoch 3/300
156/156 [==============================] - 90s 574ms/step - loss: 0.5312 - binary_accuracy: 0.7502 - val_loss: 0.4892 - val_binary_accuracy: 0.7947

Epoch 0

Fold 1
gen cache


## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 69380 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 3476 images
Epoch 1/300
156/156 [==============================] - 106s 683ms/step - loss: 1.3310 - binary_accuracy: 0.6537 - val_loss: 0.5424 - val_binary_accuracy: 0.7586

Epoch 00001: val_loss improved from inf to 0.54235, saving model to logs/boat_detecotr/best_DenseNet1691.best.hdf5
Epoch 2/300
156/156 [==============================] - 89s 571ms/step - loss: 0.6556 - binary_accuracy: 0.7248 - val_loss: 0.4841 - val_binary_accuracy: 0.7949

Epoch 00002: val_loss improved from 0.54235 to 0.48406, saving model to logs/boat_detecotr/best_DenseNet1691.best.hdf5
Epoch 3/300
156/156 [==============================] - 89s 569ms/step - loss: 0.5354 - binary_accuracy: 0.7592 - val_loss: 0.4607 - val_binary_accuracy: 0.8012

Epoch 0

## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 69384 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 3476 images
Epoch 1/300
156/156 [==============================] - 105s 675ms/step - loss: 1.1385 - binary_accuracy: 0.6485 - val_loss: 0.5111 - val_binary_accuracy: 0.7635

Epoch 00001: val_loss improved from inf to 0.51109, saving model to logs/boat_detecotr/best_DenseNet1692.best.hdf5
Epoch 2/300
156/156 [==============================] - 90s 577ms/step - loss: 0.6211 - binary_accuracy: 0.7257 - val_loss: 0.4797 - val_binary_accuracy: 0.7753

Epoch 00002: val_loss improved from 0.51109 to 0.47972, saving model to logs/boat_detecotr/best_DenseNet1692.best.hdf5
Epoch 3/300
156/156 [==============================] - 90s 578ms/step - loss: 0.5355 - binary_accuracy: 0.7563 - val_loss: 0.4472 - val_binary_accuracy: 0.7983

Epoch 0

In [15]:
from keras import backend as K
K.clear_session()

#del img_train,img_val,img_hold,train_y,val_y,hold_y
del img_val,img_hold,val_y,hold_y
gc.collect()

NameError: name 'img_val' is not defined

In [ ]:
plt.hist(oof_preds[val_df['index']])

In [ ]:
plt.hist(cur_pred)